In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn
from imblearn.over_sampling import SMOTE

In [12]:
df = pd.read_csv('investigation_train_large_checked.csv')
data_description = pd.read_csv('data_description.csv', encoding='latin1')
target = df['checked']
features = df.drop(columns=['Ja', 'Nee','checked'])
features.head()

,adres_aantal_brp_adres,adres_aantal_verschillende_wijken,adres_aantal_verzendadres,adres_aantal_woonadres_handmatig,adres_dagen_op_adres,adres_recentst_onderdeel_rdam,adres_recentste_buurt_groot_ijsselmonde,adres_recentste_buurt_nieuwe_westen,adres_recentste_buurt_other,adres_recentste_buurt_oude_noorden,...,typering_dagen_som,typering_hist_aantal,typering_hist_inburgeringsbehoeftig,typering_hist_ind,typering_hist_sector_zorg,typering_ind,typering_indicatie_geheime_gegevens,typering_other,typering_transport__logistiek___tuinbouw,typering_zorg__schoonmaak___welzijn
0,1,1,0,0,23240,1,0,0,0,0,...,-644,1,0,1,0,0,0,0,0,0
1,4,2,1,1,1971,1,0,0,1,0,...,3875,1,0,1,0,1,0,1,0,0
2,6,4,2,1,7247,0,0,0,1,0,...,3398,1,0,1,0,1,0,0,0,0
3,3,2,0,1,8060,1,0,0,1,0,...,128,1,0,1,0,0,0,0,0,0
4,3,2,0,0,18705,1,0,0,0,0,...,2210,1,0,1,0,1,0,0,0,0


In [3]:
checked_true = target.sum()
checked_false= len(target)-checked_true
print(len(target))
print('Checked == True: ',(checked_true/len(target))*100,'%')
print('Checked == False: ', (checked_false/len(target))*100,'%')
print(df.isnull().values.any())

130000
Checked == True:  15.003076923076922 %
Checked == False:  84.99692307692308 %
False


In [18]:
def get_protected_feature_indices():
    protected_indices = {
        'demographic': {
            'gender': [215],
            'age': [216],
            'language': [244,245,246,247]
        },
        'location': {
            'neighborhood': list(range(6,11)),
            'district': list(range(13,22)),
            'rotterdam': [11,12]
        },
        'family': {
            'children': list(range(282,289)),
            'partner': [302,303]
        },
        'health': {
            'physical': [54],
            'mental': [55],
            'medical': [67,68,69]
        },
        'financial': {
            'problems': [53,56],
            'income': [132,133,134]
        }
    }
    return protected_indices

def get_feature_importances():
    feature_importances = data_description['Relative importance']
    return feature_importances

def analyze_correlation_protected_features(data):
    protected_indices = get_protected_feature_indices()
    importances = get_feature_importances()
    results = {}

    for category, subcategories in protected_indices.items():
        results[category] = {}
        for subcategory, indices in subcategories.items():
            subcategory_results = []

            for idx in indices:
                protected_feature = data.columns[idx]

                for col_idx, col in enumerate(data.columns):
                    if col_idx == idx:
                        continue

                    correlation = data[protected_feature].corr(data[col], method='spearman')
                    importance = importances[col_idx]
                    risk_score = abs(correlation) * importance

                    if pd.notna(correlation):
                        subcategory_results.append({
                            'protected_feature': protected_feature,
                            'analyzed_feature': col,
                            'correlation': correlation,
                            'importance': importance,
                            'risk_score': risk_score
                        })

            subcategory_df = pd.DataFrame(subcategory_results)
            subcategory_df = subcategory_df.sort_values('risk_score',
                                                       ascending=False)

            results[category][subcategory] = subcategory_df

    return results

display_correlations = analyze_correlation_protected_features(features)
print(display_correlations)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pandas/core/nanops.py:1632: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return spearmanr(a, b)[0]


{'demographic': {'gender':           protected_feature  \
284  persoon_geslacht_vrouw   
282  persoon_geslacht_vrouw   
4    persoon_geslacht_vrouw   
274  persoon_geslacht_vrouw   
283  persoon_geslacht_vrouw   
..                      ...   
120  persoon_geslacht_vrouw   
119  persoon_geslacht_vrouw   
277  persoon_geslacht_vrouw   
223  persoon_geslacht_vrouw   
312  persoon_geslacht_vrouw   

                                      analyzed_feature  correlation  \
284   relatie_kind_leeftijd_verschil_ouder_eerste_kind     0.372755   
282                        relatie_kind_huidige_aantal     0.260127   
4                                 adres_dagen_op_adres     0.101576   
274                          pla_historie_ontwikkeling     0.110950   
283                         relatie_kind_jongvolwassen     0.227591   
..                                                 ...          ...   
120      contacten_onderwerp_boolean_taaleis___voldoet     0.016378   
119           contacten_onderwer